In [1]:
require(data.table)
require(pbapply)
pboptions(type="timer")

Loading required package: data.table

Loading required package: pbapply



In [2]:
root_path = "/mnt/data/Imaging/202105-Deconwolf/data_210726"

# Read dot tables

In [3]:
read_dots = function(dpath, magnification, image_type, nfields, ncores) {
    print(sprintf("%s - %s", magnification, image_type))
    d = rbindlist(pblapply(1:nfields, function(sid) {
        fpath = file.path(dpath, sprintf("%s_%s_calc", magnification, image_type), sprintf("%03d_allDots_snr2.dilated_labels_watershed.csv", sid))
        stopifnot(file.exists(fpath))
        d = fread(fpath)
        d$series_id = sid
        return(d)
    }, cl=ncores))
    d$magnification = magnification
    d$image_type = image_type
    return(d)
}

In [4]:
dots_data = rbindlist(list(
    read_dots(root_path, "20x", "dw", 7, 7),
    read_dots(root_path, "20x", "raw", 7, 7),
    read_dots(root_path, "60x", "dw", 7, 7),
    read_dots(root_path, "60x", "raw", 7, 7)
))

[1] "20x - dw"
  |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=05s  
[1] "20x - raw"
  |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=05s  
[1] "60x - dw"
  |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=05s  
[1] "60x - raw"
  |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=05s  


### Remove redundat or useless columns

In [5]:
dots_data[, V1 := NULL]
dots_data[, File := NULL]

### Rescale intensity based on dw scaling factor

In [6]:
scaling_values = fread("../../data/dw_scaling_factors.tsv", key=c("magnification", "image_type", "series_id"))
setkeyv(dots_data, c("magnification", "image_type", "series_id"))
dots_data2 = scaling_values[dots_data]
dots_data2[, Value2 := Value/scaling]

### Save as RDS

In [7]:
saveRDS(dots_data2, file.path(root_path, "dots_data.rds"))